# Dynamic Iceberg Tables Demo

This notebook demonstrates **Dynamic Iceberg Tables** in Snowflake - combining the power of Dynamic Tables with the Apache Iceberg format.

## What You'll Learn
- Creating Snowflake-managed Iceberg tables
- Building Dynamic Iceberg Tables for automated data pipelines
- Using TARGET_LAG for data freshness control
- Applying IMMUTABLE WHERE for optimized incremental refreshes
- Monitoring and operating Dynamic Tables

## Prerequisites
- SYSADMIN role (or equivalent privileges)
- Access to AWS S3 storage
- Storage integration configured

---
## 1. Setup Infrastructure

Create the database, schema, warehouse, and external volume for the demo.

In [ ]:
USE ROLE SYSADMIN;

-- Create dedicated database for demo
CREATE DATABASE IF NOT EXISTS DT_ICE_DEMO;
CREATE SCHEMA IF NOT EXISTS DT_ICE_DEMO.DEMO;

-- Create warehouse for demo
CREATE WAREHOUSE IF NOT EXISTS DT_ICE_WH
    WAREHOUSE_SIZE = 'X-SMALL'
    AUTO_SUSPEND = 60
    AUTO_RESUME = TRUE
    INITIALLY_SUSPENDED = TRUE;

-- Set working context
USE DATABASE DT_ICE_DEMO;
USE SCHEMA DEMO;
USE WAREHOUSE DT_ICE_WH;

### Create External Volume

External volumes define where Iceberg table data is stored in cloud storage.

> **Note:** Update the `STORAGE_BASE_URL` and `STORAGE_AWS_ROLE_ARN` with your values.

In [ ]:
CREATE OR REPLACE EXTERNAL VOLUME dt_ice_ext_volume
    STORAGE_LOCATIONS = (
        (
            NAME = 'dt_ice_s3_location'
            STORAGE_PROVIDER = 'S3'
            STORAGE_BASE_URL = 'S3 bucket'
            STORAGE_AWS_ROLE_ARN = '<your-iam-role-arn>'
        )
    )
    ALLOW_WRITES = TRUE;

In [ ]:
-- Verify setup
SHOW DATABASES LIKE 'DT_ICE_DEMO';
SHOW WAREHOUSES LIKE 'DT_ICE_WH';
DESC EXTERNAL VOLUME dt_ice_ext_volume;

---
## 2. Create Source Iceberg Tables

Create staging tables stored in Iceberg format with sample data.

In [ ]:
-- Products reference table
CREATE OR REPLACE ICEBERG TABLE products_staging (
    product_id NUMBER(10,0),
    product_name STRING,
    category STRING,
    unit_price NUMBER(10,2),
    created_at TIMESTAMP_NTZ
)
    EXTERNAL_VOLUME = 'dt_ice_ext_volume'
    CATALOG = 'SNOWFLAKE'
    BASE_LOCATION = 'products_staging/';

In [ ]:
-- Insert sample products
INSERT INTO products_staging (product_id, product_name, category, unit_price, created_at)
VALUES 
    (1, 'Laptop Pro 15', 'Electronics', 1299.99, CURRENT_TIMESTAMP()),
    (2, 'Wireless Mouse', 'Electronics', 49.99, CURRENT_TIMESTAMP()),
    (3, 'USB-C Hub', 'Electronics', 79.99, CURRENT_TIMESTAMP()),
    (4, 'Standing Desk', 'Furniture', 599.99, CURRENT_TIMESTAMP()),
    (5, 'Ergonomic Chair', 'Furniture', 449.99, CURRENT_TIMESTAMP()),
    (6, 'Monitor 27 inch', 'Electronics', 399.99, CURRENT_TIMESTAMP()),
    (7, 'Mechanical Keyboard', 'Electronics', 149.99, CURRENT_TIMESTAMP()),
    (8, 'Desk Lamp', 'Furniture', 89.99, CURRENT_TIMESTAMP()),
    (9, 'Webcam HD', 'Electronics', 129.99, CURRENT_TIMESTAMP()),
    (10, 'Cable Management Kit', 'Accessories', 29.99, CURRENT_TIMESTAMP());

SELECT * FROM products_staging ORDER BY product_id;

In [ ]:
-- Orders transactional table
CREATE OR REPLACE ICEBERG TABLE orders_staging (
    order_id NUMBER(10,0),
    customer_id NUMBER(10,0),
    product_id NUMBER(10,0),
    quantity NUMBER(5,0),
    order_date DATE,
    order_status STRING,
    region STRING,
    created_at TIMESTAMP_NTZ
)
    EXTERNAL_VOLUME = 'dt_ice_ext_volume'
    CATALOG = 'SNOWFLAKE'
    BASE_LOCATION = 'orders_staging/';

In [ ]:
-- Insert sample orders
INSERT INTO orders_staging (order_id, customer_id, product_id, quantity, order_date, order_status, region, created_at)
VALUES 
    (1001, 101, 1, 1, '2024-01-15', 'COMPLETED', 'NORTH', CURRENT_TIMESTAMP()),
    (1002, 102, 2, 3, '2024-01-16', 'COMPLETED', 'SOUTH', CURRENT_TIMESTAMP()),
    (1003, 103, 4, 1, '2024-01-17', 'COMPLETED', 'EAST', CURRENT_TIMESTAMP()),
    (1004, 101, 3, 2, '2024-01-18', 'COMPLETED', 'NORTH', CURRENT_TIMESTAMP()),
    (1005, 104, 5, 1, '2024-01-19', 'COMPLETED', 'WEST', CURRENT_TIMESTAMP()),
    (1006, 105, 1, 2, '2024-02-10', 'COMPLETED', 'SOUTH', CURRENT_TIMESTAMP()),
    (1007, 102, 6, 1, '2024-02-12', 'COMPLETED', 'SOUTH', CURRENT_TIMESTAMP()),
    (1008, 106, 7, 1, '2024-02-14', 'COMPLETED', 'EAST', CURRENT_TIMESTAMP()),
    (1009, 103, 2, 5, '2024-02-15', 'COMPLETED', 'EAST', CURRENT_TIMESTAMP()),
    (1010, 107, 8, 2, '2024-02-18', 'SHIPPED', 'NORTH', CURRENT_TIMESTAMP()),
    (1011, 101, 9, 1, '2024-03-01', 'COMPLETED', 'NORTH', CURRENT_TIMESTAMP()),
    (1012, 108, 10, 4, '2024-03-05', 'COMPLETED', 'WEST', CURRENT_TIMESTAMP()),
    (1013, 104, 1, 1, '2024-03-10', 'SHIPPED', 'WEST', CURRENT_TIMESTAMP()),
    (1014, 109, 4, 1, '2024-03-12', 'PENDING', 'SOUTH', CURRENT_TIMESTAMP()),
    (1015, 102, 5, 2, '2024-03-15', 'PENDING', 'SOUTH', CURRENT_TIMESTAMP());

SELECT * FROM orders_staging ORDER BY order_id;

In [ ]:
-- Verify record counts
SELECT 'products_staging' AS table_name, COUNT(*) AS row_count FROM products_staging
UNION ALL
SELECT 'orders_staging' AS table_name, COUNT(*) AS row_count FROM orders_staging;

---
## 3. Create Dynamic Iceberg Tables

Dynamic Iceberg Tables automatically transform and refresh data based on TARGET_LAG settings.

### Key Concepts
- **TARGET_LAG**: Maximum allowed data staleness (e.g., '10 minutes')
- **WAREHOUSE**: Compute resource for refresh operations
- **EXTERNAL_VOLUME**: Cloud storage location for Iceberg data

### 3.1 Order Details (Enriched)

Joins orders with products to create an enriched view. TARGET_LAG = 10 minutes.

In [ ]:
CREATE OR REPLACE DYNAMIC ICEBERG TABLE order_details_dit
(
    order_id NUMBER(10,0),
    customer_id NUMBER(10,0),
    product_id NUMBER(10,0),
    product_name STRING,
    category STRING,
    quantity NUMBER(5,0),
    unit_price NUMBER(10,2),
    total_amount NUMBER(12,2),
    order_date DATE,
    order_status STRING,
    region STRING
)
    TARGET_LAG = '10 minutes'
    WAREHOUSE = DT_ICE_WH
    EXTERNAL_VOLUME = 'dt_ice_ext_volume'
    CATALOG = 'SNOWFLAKE'
    BASE_LOCATION = 'order_details_dit/'
AS
SELECT 
    o.order_id,
    o.customer_id,
    o.product_id,
    p.product_name,
    p.category,
    o.quantity,
    p.unit_price,
    (o.quantity * p.unit_price) AS total_amount,
    o.order_date,
    o.order_status,
    o.region
FROM orders_staging o
JOIN products_staging p ON o.product_id = p.product_id;

### 3.2 Product Sales Summary

Aggregates sales metrics by product. TARGET_LAG = 20 minutes.

In [ ]:
CREATE OR REPLACE DYNAMIC ICEBERG TABLE product_sales_summary_dit
(
    product_id NUMBER(10,0),
    product_name STRING,
    category STRING,
    total_orders NUMBER(10,0),
    total_quantity_sold NUMBER(10,0),
    total_revenue NUMBER(14,2),
    avg_order_quantity NUMBER(10,2),
    first_order_date DATE,
    last_order_date DATE
)
    TARGET_LAG = '20 minutes'
    WAREHOUSE = DT_ICE_WH
    EXTERNAL_VOLUME = 'dt_ice_ext_volume'
    CATALOG = 'SNOWFLAKE'
    BASE_LOCATION = 'product_sales_summary_dit/'
AS
SELECT 
    p.product_id,
    p.product_name,
    p.category,
    COUNT(DISTINCT o.order_id) AS total_orders,
    SUM(o.quantity) AS total_quantity_sold,
    SUM(o.quantity * p.unit_price) AS total_revenue,
    AVG(o.quantity) AS avg_order_quantity,
    MIN(o.order_date) AS first_order_date,
    MAX(o.order_date) AS last_order_date
FROM products_staging p
LEFT JOIN orders_staging o ON p.product_id = o.product_id
GROUP BY p.product_id, p.product_name, p.category;

### 3.3 Regional Sales Dashboard

Regional performance metrics with top category calculation. TARGET_LAG = 30 minutes.

In [ ]:
CREATE OR REPLACE DYNAMIC ICEBERG TABLE regional_sales_dit
(
    region STRING,
    order_month DATE,
    total_orders NUMBER(10,0),
    total_revenue NUMBER(14,2),
    unique_customers NUMBER(10,0),
    top_category STRING,
    avg_order_value NUMBER(12,2)
)
    TARGET_LAG = '30 minutes'
    WAREHOUSE = DT_ICE_WH
    EXTERNAL_VOLUME = 'dt_ice_ext_volume'
    CATALOG = 'SNOWFLAKE'
    BASE_LOCATION = 'regional_sales_dit/'
AS
WITH regional_metrics AS (
    SELECT 
        o.region,
        DATE_TRUNC('MONTH', o.order_date) AS order_month,
        COUNT(DISTINCT o.order_id) AS total_orders,
        SUM(o.quantity * p.unit_price) AS total_revenue,
        COUNT(DISTINCT o.customer_id) AS unique_customers,
        AVG(o.quantity * p.unit_price) AS avg_order_value
    FROM orders_staging o
    JOIN products_staging p ON o.product_id = p.product_id
    GROUP BY o.region, DATE_TRUNC('MONTH', o.order_date)
),
category_revenue AS (
    SELECT 
        o.region,
        DATE_TRUNC('MONTH', o.order_date) AS order_month,
        p.category,
        SUM(o.quantity * p.unit_price) AS category_revenue,
        ROW_NUMBER() OVER (
            PARTITION BY o.region, DATE_TRUNC('MONTH', o.order_date) 
            ORDER BY SUM(o.quantity * p.unit_price) DESC
        ) AS rn
    FROM orders_staging o
    JOIN products_staging p ON o.product_id = p.product_id
    GROUP BY o.region, DATE_TRUNC('MONTH', o.order_date), p.category
)
SELECT 
    rm.region,
    rm.order_month,
    rm.total_orders,
    rm.total_revenue,
    rm.unique_customers,
    cr.category AS top_category,
    rm.avg_order_value
FROM regional_metrics rm
LEFT JOIN category_revenue cr 
    ON rm.region = cr.region 
    AND rm.order_month = cr.order_month 
    AND cr.rn = 1;

In [ ]:
-- Verify Dynamic Iceberg Tables created
SHOW DYNAMIC TABLES IN SCHEMA DT_ICE_DEMO.DEMO;
SHOW ICEBERG TABLES IN SCHEMA DT_ICE_DEMO.DEMO;

---
## 4. Query & Monitor Dynamic Tables

Explore the transformed data and monitor refresh operations.

In [ ]:
-- Query enriched order details
SELECT * FROM order_details_dit
ORDER BY order_date DESC, order_id
LIMIT 10;

In [ ]:
-- Query product sales summary
SELECT 
    product_name,
    category,
    total_orders,
    total_quantity_sold,
    total_revenue,
    ROUND(total_revenue / NULLIF(total_orders, 0), 2) AS revenue_per_order
FROM product_sales_summary_dit
ORDER BY total_revenue DESC;

In [ ]:
-- Query regional sales dashboard
SELECT 
    region,
    TO_CHAR(order_month, 'YYYY-MM') AS month,
    total_orders,
    total_revenue,
    unique_customers,
    top_category,
    ROUND(avg_order_value, 2) AS avg_order_value
FROM regional_sales_dit
ORDER BY order_month DESC, total_revenue DESC;

In [ ]:
-- Check refresh history
SELECT *
FROM TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY(
    NAME => 'DT_ICE_DEMO.DEMO.ORDER_DETAILS_DIT'
))
ORDER BY REFRESH_START_TIME DESC
LIMIT 5;

### Simulate Data Changes & Manual Refresh

In [ ]:
-- Insert new orders
INSERT INTO orders_staging (order_id, customer_id, product_id, quantity, order_date, order_status, region, created_at)
VALUES 
    (1016, 110, 1, 2, CURRENT_DATE(), 'PENDING', 'NORTH', CURRENT_TIMESTAMP()),
    (1017, 111, 6, 1, CURRENT_DATE(), 'PENDING', 'EAST', CURRENT_TIMESTAMP()),
    (1018, 112, 3, 3, CURRENT_DATE(), 'PENDING', 'WEST', CURRENT_TIMESTAMP());

-- Trigger refresh to see changes
ALTER DYNAMIC TABLE order_details_dit REFRESH;

-- Verify new data
SELECT * FROM order_details_dit WHERE order_date = CURRENT_DATE();

---
## 5. IMMUTABLE WHERE - Optimized Incremental Refresh

The `IMMUTABLE WHERE` clause marks certain rows as unchangeable, allowing Snowflake to skip re-processing them during incremental refreshes.

### Rules for IMMUTABLE WHERE
- Predicates must only **GROW** the immutable region over time
- ❌ `order_date < CURRENT_DATE() - 7` (shrinks as time passes - NOT ALLOWED)
- ✅ `order_status = 'COMPLETED'` (only grows as orders complete)
- ✅ `order_date < '2024-01-01'` (fixed cutoff, never changes)

### 5.1 Status-Based Immutability

Completed orders won't change - skip them during refresh.

In [ ]:
CREATE OR REPLACE DYNAMIC ICEBERG TABLE order_analytics_immutable_dit
(
    order_id NUMBER(10,0),
    customer_id NUMBER(10,0),
    product_id NUMBER(10,0),
    product_name STRING,
    category STRING,
    quantity NUMBER(5,0),
    unit_price NUMBER(10,2),
    total_amount NUMBER(12,2),
    order_date DATE,
    order_status STRING,
    region STRING,
    is_immutable BOOLEAN
)
    TARGET_LAG = '5 minutes'
    WAREHOUSE = DT_ICE_WH
    EXTERNAL_VOLUME = 'dt_ice_ext_volume'
    CATALOG = 'SNOWFLAKE'
    BASE_LOCATION = 'order_analytics_immutable_dit/'
    IMMUTABLE WHERE (order_status = 'COMPLETED')
AS
SELECT 
    o.order_id,
    o.customer_id,
    o.product_id,
    p.product_name,
    p.category,
    o.quantity,
    p.unit_price,
    (o.quantity * p.unit_price) AS total_amount,
    o.order_date,
    o.order_status,
    o.region,
    CASE WHEN o.order_status = 'COMPLETED' THEN TRUE ELSE FALSE END AS is_immutable
FROM orders_staging o
JOIN products_staging p ON o.product_id = p.product_id;

In [ ]:
-- Check data breakdown by status
SELECT 
    order_status,
    is_immutable,
    COUNT(*) AS order_count
FROM order_analytics_immutable_dit
GROUP BY order_status, is_immutable
ORDER BY order_status;

### 5.2 Test Incremental Refresh

In [ ]:
-- Insert new pending orders
INSERT INTO orders_staging (order_id, customer_id, product_id, quantity, order_date, order_status, region, created_at)
VALUES 
    (1020, 120, 1, 1, CURRENT_DATE(), 'PENDING', 'NORTH', CURRENT_TIMESTAMP()),
    (1021, 121, 5, 2, CURRENT_DATE(), 'PENDING', 'EAST', CURRENT_TIMESTAMP());

-- Refresh - completed orders will be skipped
ALTER DYNAMIC TABLE order_analytics_immutable_dit REFRESH;

-- Check new orders
SELECT * FROM order_analytics_immutable_dit 
WHERE order_status = 'PENDING'
ORDER BY order_id DESC;

In [ ]:
-- Complete an order - it becomes immutable
UPDATE orders_staging 
SET order_status = 'COMPLETED'
WHERE order_id = 1020;

ALTER DYNAMIC TABLE order_analytics_immutable_dit REFRESH;

-- Verify status change
SELECT order_id, order_status, is_immutable 
FROM order_analytics_immutable_dit 
WHERE order_id IN (1020, 1021);

### 5.3 Fixed Date Cutoff Immutability

In [ ]:
CREATE OR REPLACE DYNAMIC ICEBERG TABLE sales_fixed_cutoff_dit
(
    region STRING,
    order_month DATE,
    total_orders NUMBER(10,0),
    total_revenue NUMBER(14,2)
)
    TARGET_LAG = '10 minutes'
    WAREHOUSE = DT_ICE_WH
    EXTERNAL_VOLUME = 'dt_ice_ext_volume'
    CATALOG = 'SNOWFLAKE'
    BASE_LOCATION = 'sales_fixed_cutoff_dit/'
    IMMUTABLE WHERE (order_month < '2024-01-01')
AS
SELECT 
    o.region,
    DATE_TRUNC('MONTH', o.order_date)::DATE AS order_month,
    COUNT(*) AS total_orders,
    SUM(o.quantity * p.unit_price) AS total_revenue
FROM orders_staging o
JOIN products_staging p ON o.product_id = p.product_id
GROUP BY o.region, DATE_TRUNC('MONTH', o.order_date);

SELECT * FROM sales_fixed_cutoff_dit ORDER BY order_month;

---
## Summary

### Key Takeaways

1. **Dynamic Iceberg Tables** combine automated data pipelines with the Iceberg format
2. **TARGET_LAG** controls data freshness (e.g., '10 minutes', '1 hour')
3. **IMMUTABLE WHERE** optimizes incremental refreshes by skipping unchanged data
4. Predicates must only **GROW** the immutable region over time

### Limitations
- `IF NOT EXISTS` clause not supported
- Cannot clone Dynamic Iceberg Tables
- ALTER IMMUTABLE not supported for Dynamic Iceberg Tables
- BACKFILL FROM not supported for Dynamic Iceberg Tables

---
**Next:** Run `cleanup.ipynb` when ready to remove all demo objects.